In [13]:
__author__ = "nickirom"

import requests
from bs4 import BeautifulSoup
from pprint import pprint
from urlparse import urljoin
import time
from datetime import datetime, timedelta
from IPython.display import Image
import re
import random
import sys


#CURRENT_TIME = time.strftime("%Y-%m-%d %H:%M")
CURRENT_TIME = datetime.now()
BATCH_MIN_TIME = CURRENT_TIME - timedelta(hours=1)

MAX_WAIT = 3

BASE_URL = 'http://sfbay.craigslist.org/search/sfc/roo?s=0'

In [22]:
def get_all_listings(url):
    run_again = 1
    all_apts = []

    while run_again == 1:
        apts, run_again, url = get_listings(url, run_again)
        all_apts.extend(apts)
    
    return all_apts
        
def scrape_all_apts(listings):
    apt_data = []
    for apt in listings:
        link = apt.find('a').attrs['href']

        # join this relative link with the
        # BASE_URL to create an absolute link
        url = urljoin(BASE_URL, link)

        apt_data.append(scrape_apt(url)[1])
        
    return apt_data

def get_listings(url, run_again):
    response = requests.get(url)
    
    soup = BeautifulSoup(response.content.decode('utf-8', 'ignore'))
    apts = soup.find_all('span', {'class':'pl'})

    min_time = datetime.strptime(min([item.time.attrs['datetime'] for item in apts]), "%Y-%m-%d %H:%M")

    if min_time < BATCH_MIN_TIME:
        run_again = 0
        apts = [apt for apt in apts if datetime.strptime(apt.time.attrs['datetime'], "%Y-%m-%d %H:%M") >= BATCH_MIN_TIME]

    else: 
        start_num = str(int(re.findall('\?s=(.+)', url)[0]) + 100)
        url = re.findall('(.+)\?s=', url)[0] + "?s=" + start_num
    
    time.sleep(MAX_WAIT*random.random())

    return apts, run_again, url

def scrape_apt(url):
    """ Extract information from a apt page. """

    print "scraping url %s" %str(url)
    response = requests.get(url)
    
    # Parse the html of the  post
    soup = BeautifulSoup(response.content.decode('utf-8', 'ignore'))

    # Extract the actual contents of some HTML elements:
    
    data = {
        'source_url': url,
        'subject': None,
        'price':  None,
        'neighborhood': None,
        'sqft': None,
        'tags': [],
        'images': [],
        'geo': None,
        'body': ''
    }
    
    try: data['subject'] = get_data(soup.find('h2', {'class':'postingtitle'}).find('span', {'id':'titletextonly'}), 'subject')
    except: data['subject'] = None
        
    try: data['price'] = get_data(soup.find('span', {'class': 'price'}), 'price')
    except: data['price'] = None

    try: data['neighborhood'] = neighborhood(get_data(soup.find('small'), 'neighborhood').strip())
    except: data['neighborhood'] = None
    
    try: data['sqft'] = sqft(get_data(soup.find('span', {'class': 'housing'}), 'sqft'))
    except: pass
    
    try: data['tags'] = tags(soup('p', {'class':'attrgroup'})[1])
    except: pass
    
    try: data['images'] = get_images(soup)
    except: pass
    
    try: data['geo'] = get_geo(soup),
    except: pass
    
    try: data['body']= get_data(soup.find('section', {'id':'postingbody'}), 'body')
    
        # Print it prettily.
        #pprint([data[key] for key in data.keys() if key != 'body'])
    
    except UnicodeEncodeError as e:
        print url
        print "Unicode Error: ", e
        data = url

    except AttributeError as e:
        print url
        print "AttributeError! : ", e
        data = e
    else:
        print sys.exc_info()[0]
    

    
    time.sleep(MAX_WAIT*random.random())
    
    return soup, data

def get_data(action, name):
    try:
        result = action
        return result.get_text()
    except AttributeError as e:
        print "Error getting data"
        print "Action was: %s" %str(name)
        return None

def subject(action):
    print str(action)
    
def price(action):
    print str(action)
    
def tags(action):
    try:
        tags =[p.text for p in action if len(p)>0]
        return tags
    except AttributeError:
        print "tag AttributeError"
        return None
    
def price(text):
    try:
        price = int(re.search('\$\s*([0-9]+)', text).groups()[0])
    except AttributeError:
        print "pricing AttributeError"
        price = None
    except TypeError:
        print "price data DNE"
        price = None
    return price
    
def sqft(text):
    try:
        sqft = int(re.search('\/\s*([0-9]+)', text).groups()[0])   
    except AttributeError:
        print "sqft AttributeError"
        sqft= None
    except TypeError:
        print "sqft data DNE"
        sqft= None
    return sqft

def neighborhood(text):
    try:
        hood = re.search('\s*\(\s*(.*)\s*\)\s*', text).groups()[0]   
    except AttributeError:
        print "neighborhood AttributeError"
        hood= None
    except TypeError:
        print "neighborhood data DNE"
        hood= None
    return hood

def get_images(soup):
    
    try:
        #check for thumbnails due to multiple images:
        urls =[thumb.find('img')['src'] for thumb in soup.find('div', {'id':'thumbs'})]
    except:
        try:
            urls = [get_data(soup.find('div', {'class': 'slide first visible'}).find('img')['src'], 'image')]
        except:
            urls = None
    return urls

def get_geo(soup):
    #check for thumbnails due to multiple images:
    try:
        div = soup.find('div', {'class':'mapbox'}).find('div', {'class':'viewposting'})
        geo = {'lat':div['data-latitude'], 'lon':div['data-longitude']}
    except:
        geo= None
    return geo

def room_dimensions(body):
    try: 
        dims = [int(dim) for dim in re.match(".*[\s|\(]{1}([0-9]{2})[']*[\s]*[x]\s*([0-9]{2})[']*[\s|\)]{1}.*", body).groups()]
        return dims
    except AttributeError:
        print "room_dimensions AttributeError"
        return None

In [23]:
def run(url):

    listings = get_all_listings(url)
    print "listings have been retrieved"
    
    apt_data = scrape_all_apts(listings)
    
    return apt_data
    

In [24]:
apt_data = run(BASE_URL)

listings have been retrieved
scraping url http://sfbay.craigslist.org/sfc/roo/5492851443.html
Error getting data
Action was: sqft
sqft data DNE
Error getting data
Action was: image
None
scraping url http://sfbay.craigslist.org/sfc/roo/5500712315.html
Error getting data
Action was: sqft
sqft data DNE
None
scraping url http://sfbay.craigslist.org/sfc/roo/5494310057.html
Error getting data
Action was: sqft
sqft data DNE
None
scraping url http://sfbay.craigslist.org/sfc/roo/5494458355.html
None
scraping url http://sfbay.craigslist.org/sfc/roo/5494465513.html
Error getting data
Action was: sqft
sqft data DNE
None
scraping url http://sfbay.craigslist.org/sfc/roo/5494397090.html
Error getting data
Action was: sqft
sqft data DNE
None
scraping url http://sfbay.craigslist.org/sfc/roo/5494396002.html
Error getting data
Action was: sqft
sqft data DNE
None
scraping url http://sfbay.craigslist.org/sfc/roo/5494308149.html
None
scraping url http://sfbay.craigslist.org/sfc/roo/5484285255.html
Error get

In [ ]:
for apt in apt_data:
    print [(key,apt[key]) for key in apt.keys() if key !='body']
    print '\n'

In [ ]:
url = 'http://sfbay.craigslist.org/sfc/roo/5472543000.html'
response = requests.get(url)
print url 
print response

soup = BeautifulSoup(response.content.decode('utf-8', 'ignore'))
#print [thumb.find('img')['src'] for thumb in soup.find('div', {'id':'thumbs'})]
#print get_data(soup.find('span', {'class': 'price'}), 'price')
print sqft(get_data(soup.find('span', {'class': 'housing'}), 'sqft'))
price(soup)

In [25]:
import pandas as pd

df = pd.DataFrame.from_dict(apt_data)

In [26]:
df.head()

,body,geo,images,neighborhood,price,source_url,sqft,subject,tags
0,"\nHuge, amazing apartment on Union St right ne...","({u'lat': u'37.796879', u'lon': u'-122.437138'},)",[None],marina / cow hollow,$1750,http://sfbay.craigslist.org/sfc/roo/5492851443...,NaN,Great Deal! Furnished Amazing Apt on Union,"[furnished, no smoking, private room, private ..."
1,"\nRoom available ASAP.\n\nHi, my name is Guido...","({u'lat': u'37.713210', u'lon': u'-122.404990'},)",[http://images.craigslist.org/00808_czHZfoh7JC...,visitacion valley,$1000,http://sfbay.craigslist.org/sfc/roo/5500712315...,NaN,1 bedroom in 2br apt,"[no smoking, private room, no private bath, w/..."
2,"\nLocated in the highly desired SOMA area, bet...","({u'lat': u'37.777726', u'lon': u'-122.409072'},)",[http://images.craigslist.org/00E0E_cS2ZIgvJpB...,SOMA / south beach,$900,http://sfbay.craigslist.org/sfc/roo/5494310057...,NaN,"Amazing Shared Room, Mission District","[furnished, no smoking, room not private, no p..."
3,"\n1010 Bush Street, San Francisco -- Balmoral ...","({u'lat': u'37.789509', u'lon': u'-122.413888'},)",[http://images.craigslist.org/00p0p_iIpczMcRsD...,financial district,$1795,http://sfbay.craigslist.org/sfc/roo/5494458355...,108,"Private Rooms, Great Location","[furnished, no smoking, room not private, no p..."
4,\nLocated in the highly desired SOMA area - co...,"({u'lat': u'37.777100', u'lon': u'-122.407394'},)",[http://images.craigslist.org/00f0f_3E6N5oQvla...,mission district,$650,http://sfbay.craigslist.org/sfc/roo/5494465513...,NaN,QUAD available in dorm style accommodation,"[furnished, room not private, no private bath,..."


In [27]:
df['room_dimensions'] = df['body'].apply(lambda x: room_dimensions(x))

room_dimensions AttributeError
room_dimensions AttributeError
room_dimensions AttributeError
room_dimensions AttributeError
room_dimensions AttributeError
room_dimensions AttributeError
room_dimensions AttributeError
room_dimensions AttributeError
room_dimensions AttributeError
room_dimensions AttributeError
room_dimensions AttributeError
room_dimensions AttributeError
room_dimensions AttributeError
room_dimensions AttributeError
room_dimensions AttributeError
room_dimensions AttributeError
room_dimensions AttributeError
room_dimensions AttributeError
room_dimensions AttributeError
room_dimensions AttributeError
room_dimensions AttributeError
room_dimensions AttributeError
room_dimensions AttributeError
room_dimensions AttributeError
room_dimensions AttributeError
room_dimensions AttributeError
room_dimensions AttributeError
room_dimensions AttributeError
room_dimensions AttributeError
room_dimensions AttributeError
room_dimensions AttributeError
room_dimensions AttributeError
room_dim

In [ ]:
room_dimensions(df.iloc[0,0])